In [1]:
import pandas as pd

# Read the CSV file
data = pd.read_csv("pred2017l.csv")

# Display the data
data.head()

,Open,High,Low,Close,HOD,LOD,HOY,LOY,VPOC,VPOC-1,LinReg,LinRegSlope,CumDelta,Ticks
0,-0.02,-0.04,0.10,0.04,-0.010,0.025,-0.140,0.260,0.04,0.00,1.00,0.00,-134,0.06
1,0.20,-0.06,0.24,0.00,-0.015,0.075,-0.095,0.305,0.04,0.24,0.84,0.36,2567,-0.04
2,0.06,-0.06,0.06,0.00,-0.015,0.085,-0.085,0.315,-0.02,0.08,0.86,0.49,-186,0.00
3,0.02,0.00,0.10,0.04,-0.015,0.085,-0.085,0.315,0.04,0.06,0.76,0.62,-376,0.00
4,0.04,-0.06,0.06,0.00,-0.015,0.105,-0.065,0.335,-0.02,0.10,0.74,0.60,386,0.10


Train-valid-test split

In [2]:
from sklearn.model_selection import train_test_split

# train-valid-test split
X_temp = data.drop('Ticks',axis=1)
y_temp = data['Ticks']

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, train_size=0.75)

X_test  = data.drop('Ticks',axis=1)
y_test  = data['Ticks']

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)

(2136, 13)
(2136,)
(713, 13)
(713,)
(2849, 13)


Convert data into XGB format

In [3]:
import xgboost as xgb

d_train = xgb.DMatrix(X_train, y_train)
d_valid = xgb.DMatrix(X_valid, y_valid)
d_test  = xgb.DMatrix(X_test)

XGB parameters

In [4]:
xgb_params = {
    'n_trees': 75, 
    'eta': 0.03,
    'max_depth': 4,
    'subsample': 0.90,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    #'base_score': y_mean, # base prediction = mean(target)
    'silent': 0}

#
num_boost_rounds = 1000
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

Train the model

In [ ]:
# Train the model!
mdl = xgb.train(dict(xgb_params, silent=0), d_train, num_boost_rounds, watchlist, early_stopping_rounds=500, maximize=False, verbose_eval=10)

Predict the test data

In [6]:
y_pred = mdl.predict(d_test)

print(y_pred)

[ 0.05475697  0.01549684 -0.00066602 ...  0.01840273  0.00705392
  0.04463446]


test the quality of a model

In [7]:
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error
# Reshape y_pred to match the shape of y_valid
y_pred = y_pred[:len(y_valid)]

# Calculate the mean squared error
mse = mean_squared_error(y_valid, y_pred)
rmse = root_mean_squared_error(y_valid, y_pred)
mae= mean_absolute_error(y_valid, y_pred)

print('Mean Squared Error is:', mse)
print('Root Mean Squared Error is:', rmse)
print('Mean Absolute Error is:', mae)


Mean Squared Error is: 0.009798797830730425
Root Mean Squared Error is: 0.09898887730816239
Mean Absolute Error is: 0.07431107214148625
